# speed rush to overview torch.

In [57]:
import torch
import torchvision
import numpy as np
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)


mps


In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

# Load data and print as a matrix using matplot

In [59]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
for x, y in train_loader:
    print(x.shape, y.shape)
    break




torch.Size([64, 1, 28, 28]) torch.Size([64])


# FunNet

In [ ]:
class FunNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.conv = torch.nn.Conv3d()
        self.hidden = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden(x)
        return x

In [61]:
model = FunNet().to(device)
print(model)


FunNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (hidden): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# define loss function and optimizer.


In [63]:
# loss_fn 
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model: torch.nn.Module, loss_fn, optim):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        pred = model(x)

        loss = loss_fn(pred, y)
        loss.backward()
        optim.step()
        optim.zero_grad()
        if batch % 100 == 0:
            loss = loss.item()
            print(f"loss: {loss:>7f}")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
epoch = 500
for i in range(epoch):
    print(f"Epoch {i+1}\n-------------------------------")
    train(train_loader, model=model, loss_fn=loss_fn, optim=optimizer)
    test(test_loader, model=model, loss_fn=loss_fn)
    

Epoch 1
-------------------------------
loss: 1.136415
loss: 1.144183
loss: 0.957477
loss: 1.098169
loss: 0.976871
loss: 1.007914
loss: 1.046949
loss: 0.966616
loss: 1.013037
loss: 0.969064
Epoch 2
-------------------------------
loss: 1.025831
loss: 1.053030
loss: 0.849836
loss: 1.011408
loss: 0.894811
loss: 0.918665
loss: 0.975040
loss: 0.897114
loss: 0.938172
loss: 0.908071
Epoch 3
-------------------------------
loss: 0.944787
loss: 0.989560
loss: 0.772224
loss: 0.949997
loss: 0.838980
loss: 0.852976
loss: 0.923141
loss: 0.850465
loss: 0.883918
loss: 0.863105
Epoch 4
-------------------------------
loss: 0.881918
loss: 0.941607
loss: 0.713595
loss: 0.904025
loss: 0.798029
loss: 0.803263
loss: 0.883007
loss: 0.817268
loss: 0.842929
loss: 0.827559
Epoch 5
-------------------------------
loss: 0.830824
loss: 0.902675
loss: 0.667263
loss: 0.868137
loss: 0.766257
loss: 0.764459
loss: 0.849750
loss: 0.792304
loss: 0.810694
loss: 0.798222
Epoch 6
-------------------------------
loss: 0.78

KeyboardInterrupt: 